In [102]:
#Import the needed modules
import klayout as kl
import klayout.db as db
import pya
import pandas as pd

#Define the function
def rpdb(gds_file:str = 'Contact_test_tD_s100_p200_l10000_a0_3to1.gds', mesh:str = 'Test_Mesh.csv', target_layer:int = 2, LL:list=(-5,-5), UR:list=(5,5)):
    #gds is the input gds, and mesh is the matrix of data biasing to be applied, target_layer is the specific layer to be biased
    #LL is the lower left corner of the layer to be edited that contains structures, UR is the upper right corner (in um)
    #Will hardcode these for now for testing purposes

    #Import the .gds
    gds = pya.Layout()
    gds.read(gds_file)

    
    #Sets the database units as 1nm
    gds.dbu = 0.001

    #Determine number of layers for copy and bias layer indicies (based on order of creation in script)
    num_lay = len(gds.layer_indexes())
    copy_lyr_index = num_lay
    bias_lyr_index = num_lay+1

    #Create copy layer, copy structures over from the desired layer for biasing.
    #This copy layer will be mauled by the bias layer so that the original target layer is untouched.
    copy_lyr = gds.layer(98,0)
    gds.copy_layer(target_layer,copy_lyr_index)

    #Defines a bounding box for the copy layer
    global_bbox = gds.top_cell().dbbox_per_layer(copy_lyr)


    #Create bias layer for eventual use
    bias_lyr = gds.layer(99,0)

    #Import the mesh
    pdb = pd.read_csv(mesh, index_col=0)
    
    #Determine mesh columns and convert to string for DataFrame access
    col_len = len(pdb.columns)+1
    print(col_len)
    columns = range(1,col_len)
    columns_str = [str(x) for x in columns]

    #Determine mesh rows
    row_len = len(pdb)+1
    rows = range(1,row_len)

    #Determine layer sizing
    x_size = UR[0] - LL[0]
    y_size = UR[1] - LL[1]

    #Define size of meshing
    mesh_x = x_size / (len(columns))
    mesh_y = y_size / (len(rows))

    i=5
    j=5
    bias = (pdb[str(j)][i])
    print(bias)
    print(type(bias))


    top_cell = gds.top_cell()

    #Loop across the mesh, define the regions in the copy layer to copy into the bias layer, then resize the features in the bias layer based on the mesh input
    for i in rows:
        for j in columns:
                #Define the edges of the mesh at each location
                l_edge = LL[0]+mesh_x*(j-1)
                b_edge = UR[1]-mesh_y*i
                r_edge = LL[0]+mesh_x*j
                t_edge = UR[1]-mesh_y*(i-1)

                #Define the biasing at each location
                #biasing = pdb[str(j)][i]
                new_j = str(j)
                biasing = pdb[new_j][i]

                #Create the mesh cell
                mesh_cell = gds.create_cell(f'mesh_cell_{i}_{j}')
                bias_cell = gds.create_cell(f'bias_cell_{i}_{j}')

                #Defines the region, sizes it, then inserts it into the bias layer
                r1 = pya.Region(gds.begin_shapes_touching(top_cell,copy_lyr,pya.Box(l_edge, b_edge, r_edge, t_edge)))
                result1 = r1.sized(int(biasing))
                bias_insert = bias_cell.shapes(bias_lyr).insert(result1)

                #Destroys the shapes on the copy layer
                r2 = pya.Region(mesh_cell.shapes(copy_lyr))
                result2 = r1 - r2


                

                #Create the mesh cell
                #mesh_cell = gds.create_cell('mesh_cell')
                #bias_cell = gds.create_cell('bias_cell')

                #Defines a mesh shape that  the shape in the cell doing the copying
                #mesh_area = bias_cell.shapes(copy_lyr).each_touching(pya.Box(l_edge, b_edge, r_edge, t_edge))
                #bias_cell = mesh_cell.shapes(copy_lyr).each_touching(pya.Box(LL[0]+mesh_x*(j-1),UR[1]-mesh_y*i,LL[0]+mesh_x*j,UR[1]-mesh_y*(i-1)))
                #r1 = pya.Region(bias_cell.shapes(copy_lyr).each_touching(pya.Box(LL[0]+mesh_x*(j-1),UR[1]-mesh_y*i,LL[0]+mesh_x*j,UR[1]-mesh_y*(i-1))))
                #result = r1.sized(pdb[i,j])

                #print(f"Left {l_edge}, bottom {b_edge}, right {r_edge}, top {t_edge}")
                #print(f"Finished row {i} and column {j}")

    #return mesh_x, j, i, LL[0], LL[1]
    return gds.write('Test RDP.gds')




rpdb()



Finished row 1 and column 1
Finished row 1 and column 2
Finished row 1 and column 3
Finished row 1 and column 4
Finished row 1 and column 5
Finished row 1 and column 6
Finished row 1 and column 7
Finished row 1 and column 8
Finished row 1 and column 9
Finished row 1 and column 10
Finished row 1 and column 11
Finished row 1 and column 12
Finished row 1 and column 13
Finished row 1 and column 14
Finished row 1 and column 15
Finished row 2 and column 1
Finished row 2 and column 2
Finished row 2 and column 3
Finished row 2 and column 4
Finished row 2 and column 5
Finished row 2 and column 6
Finished row 2 and column 7
Finished row 2 and column 8
Finished row 2 and column 9
Finished row 2 and column 10
Finished row 2 and column 11
Finished row 2 and column 12
Finished row 2 and column 13
Finished row 2 and column 14
Finished row 2 and column 15
Finished row 3 and column 1
Finished row 3 and column 2
Finished row 3 and column 3
Finished row 3 and column 4
Finished row 3 and column 5
Finished